# 1. JDK & JRE
---

我的电脑有两个jdk, 一个是我自己下载的jdk17, 一个是电脑预安装的jdk 19:

```bash
ls /Library/Java/JavaVirtualMachines/jdk-19.jdk/Contents/Home/
LICENSE bin     include legal   man README  conf    jmods   lib     release

ls ~/Downloads/Programs/jdk-17-0-3-1/Home/
LICENSE README  bin     conf    include jmods   legal   lib     release
```

都说JDK包括JRE, 然后JRE里面有JVM, 但是现在新版本的JDK里没有JRE文件夹了, JRE被单独安装了, 那么现在对于新版本的JDK来说, JRE是不是仍然属于JDK呢? 知道这个问题和现实就行了, 至于属不属于无所谓, 想怎么说就怎么说呗, 关键是我们得是找, 什么是JVM, 什么是JDK才是重要的. 


- In macOS, the JDK installation path is/Library/Java/JavaVirtualMachines/jdk-10.jdk/Contents/Home.
- In macOS, the JRE installation path is/Library/Internet Plug-Ins/JavaAppletPlugin.plugin/Contents/Home/.

但是我去找jre却没找到, 官方说 When you install JDK 10, the public JRE (Release 10) also gets installed automatically. 他说的这种安装是下载dmg文件, 而我下的是免安装版本的zip包, 解压出来就用了, 所以ummmm, 具体也不清楚以后再想吧...



# 2. PATH Variable
---

You should set the `PATH` variable if you want to be able to run the executables (javac, java, javadoc, and so on) from any directory without having to type the full path of the command. If you do not set the PATH variable, you need to specify the full path to the executable every time you run it, such as:

```bash
/usr/local/jdk1.7.0/bin/javac MyClass.java
```

怎么设置PATH呢, 也就是win里面说的怎么添加环境变量呢? 编辑`~/.bashrc`或者`.zshrc`, 看你用的什么bash, 默认用的都是bash, 我是自己用的zsh

```
# 填进文件里,
export PATH=$PATH:/place/with/the/file
```

其实这个很容易看出什么意思, PATH = `$PATH` + `/place/with/the/file`, 橘子里面的分号就是个separator而已. 比如下面的输出, 一看就明白了:

```bash
echo $PATH
/opt/homebrew/bin:/opt/homebrew/sbin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin
```

On most systems (Linux, Mac OS, UNIX, etc) the colon character (`:`) is the classpath separator. In windowsm the separator is the semicolon (`;`)

> 其实PATH就是告诉terminal可执行指令的位置信息, 而CLASSPATH是用来告诉JRE相关程序 用户自定义类 的位置信息 

# 3. CLASSPATH
---

> The class path is the path that the Java Runtime Environment (JRE) searches for classes and other resource files. [Find More](https://docs.oracle.com/javase/8/docs/technotes/tools/windows/classpath.html)

The class search path (class path) can be set using either the `-classpath` option when calling a JDK tool (the preferred method) or by setting the `CLASSPATH` environment variable. 

The `-classpath` option is preferred because you can set it individually for each application without affecting other applications and without other applications modifying its value.


**When classes are stored in a directory (folder)**, such as `c:\java\MyClasses\utility\myapp`, then the class path entry points to the directory that contains the first element of the package name (in this case, `C:\java\MyClasses`, because the package name is `utility.myapp`).

**When classes are stored in an archive file** (a zip or JAR file) the class path entry is the path to and including the zip or JAR file. 

You can check value of classpath in java inside your application by looking at following system property “java.class.path” 
```
System.getProperty("java.class.path")
```

## 3.1. Stackoverflow 的回答

When programming in Java, you make other classes available to the class you are writing by putting something like this at the top of your source file:

```java
import org.javaguy.coolframework.MyClass;
```

Or sometimes you 'bulk import' stuff by saying:

```java
import org.javaguy.coolframework.*;
```

So later in your program when you say:
```
MyClass mine = new MyClass();
```

The Java Virtual Machine will know where to find your compiled class.

It would be impractical to have the VM look through every folder on your machine, so you have to provide the VM a list of places to look. This is done by putting folder and jar files on your classpath.

Before we talk about how the classpath is set, let's talk about `.class` files, `packages`, and `.jar` files.

First, let's suppose that `MyClass` is something you built as part of your project, and it is in a directory in your project called `greenhouse`. The .class file would be at `greenhouse/org/javaguy/coolframework/MyClass.class`, In order to get to that file, your path would simply need to contain the folder `greenhouse`, not the whole package structure, **since your import statement provides all that information** to the VM. JVM需要的是class文件而不是源码.

Now let's suppose that you bundle `CoolFramework` up into a `.jar` file, and put that `CoolFramework.jar` into a lib directory in your project. You would now need to put `lib/CoolFramework.jar` into your classpath. The VM will look inside the jar file for the `org/javaguy/coolframework` part, and find your class.

**So, classpaths contain**:

- JAR files, and
- Paths to the top of package hierarchies.

### 3.1.1 How do you set your classpath?

The **first** way everyone seems to learn is with environment variables. On a unix machine, you can say something like:

```
export CLASSPATH=$PATH:/home/myaccount/myproject/lib/CoolFramework.jar:/home/myaccount/myproject/output/
```

The **second** way is to use the -cp parameter when starting Java, like this:

```
java -cp "/home/myaccount/myproject/lib/CoolFramework.jar:/home/myaccount/myproject/output/"  MyMainClass
```

### 3.1.2. So what's the best way to do it?

Setting stuff globally via environment variables is bad, generally for the same kinds of reasons that global variables are bad. You change the `CLASSPATH` environment variable so one program works, and you end up breaking another program.

The `-cp` is the way to go. I generally make sure my `CLASSPATH` environment variable is an empty string where I develop, whenever possible, so that I avoid global classpath issues (some tools aren't happy when the global classpath is empty though - I know of two common, mega-thousand dollar licensed J2EE and Java servers that have this kind of issue with their command-line tools).

https://stackoverflow.com/a/2396759/16317008

## How the Java Launcher Finds Classes
---

The Java launcher, `java`, initiates the Java virtual machine. The virtual machine searches for and loads classes in this order:

- Bootstrap classes - Classes that comprise the Java platform, including the classes in `rt.jar` and several other important jar files.
- Extension classes - Classes that use the Java Extension mechanism. These are bundled as `.jar` files located in the extensions directory.
- User classes - Classes defined by developers and third parties that do not take advantage of the extension mechanism. You identify the location of these classes using the -classpath option on the command line (the preferred method) or by using the CLASSPATH environment variable. (See Setting the Classpath for [Windows or Unix](https://docs.oracle.com/javase/8/docs/technotes/tools/unix/classpath.html).)


https://docs.oracle.com/javase/8/docs/technotes/tools/findingclasses.html